In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gmap_key

In [6]:
# read csv

cities = os.path.join("cities.csv")
cities_df = pd.read_csv(cities)
cities_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Qaanaaq,77.4840,-69.3632,13.55,70,97,8.88,GL,1638701721
1,1,Port Alfred,-33.5906,26.8910,72.61,87,100,23.47,ZA,1638701721
2,2,Tazovsky,67.4667,78.7000,-26.66,99,74,8.03,RU,1638701721
3,3,Vaxjo,56.8777,14.8091,28.45,86,90,13.80,SE,1638701721
4,4,Luanda,-8.8368,13.2343,82.40,78,75,4.61,AO,1638701701
...,...,...,...,...,...,...,...,...,...,...
540,540,Sault Ste. Marie,46.5168,-84.3333,26.89,83,75,4.61,CA,1638701793
541,541,Tuensang,26.2833,94.8333,62.33,92,49,1.14,IN,1638701793
542,542,Bulgan,48.8125,103.5347,20.50,48,1,4.00,MN,1638701794
543,543,Komatsu,36.4026,136.4509,40.24,75,20,2.30,JP,1638701794


In [7]:
gmaps.configure(api_key=gmap_key)

In [8]:
locations = cities_df[["Latitude", "Longitude"]]
weight = cities_df["Humidity"]
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [61]:
# alter the dataframe for ideal weather conditions

ideal_df = cities_df.loc[(cities_df["Max Temperature"] >=60) & (cities_df["Max Temperature"] <=85) & (cities_df["Wind Speed"] <10) & (cities_df["Cloudiness"] == 0)]

hotel_df = ideal_df.dropna()
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,Salalah,17.0151,54.0924,83.66,34,0,8.37,OM,1638701726
43,43,Adelaide,-34.9333,138.6000,67.42,61,0,4.00,AU,1638701727
131,131,General Roca,-39.0333,-67.5833,61.20,53,0,7.49,AR,1638701738
168,168,Waddān,29.1614,16.1390,61.27,35,0,6.53,LY,1638701743
252,252,Jalu,29.0331,21.5482,68.40,41,0,8.70,LY,1638701756
336,336,Usta Muhammad,28.1794,68.0445,81.64,14,0,4.36,PK,1638701768
375,375,Constitución,-35.3333,-72.4167,62.44,57,0,3.22,CL,1638701773
381,381,Isla Vista,34.4133,-119.8610,64.11,66,0,4.18,US,1638701774
382,382,Acapulco de Juárez,16.8634,-99.8901,75.60,72,0,3.65,MX,1638701774
417,417,Sur,22.5667,59.5289,78.37,55,0,9.73,OM,1638701778


In [63]:
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
35,35,Salalah,17.0151,54.0924,83.66,34,0,8.37,OM,1638701726,
43,43,Adelaide,-34.9333,138.6000,67.42,61,0,4.00,AU,1638701727,
131,131,General Roca,-39.0333,-67.5833,61.20,53,0,7.49,AR,1638701738,
168,168,Waddān,29.1614,16.1390,61.27,35,0,6.53,LY,1638701743,
252,252,Jalu,29.0331,21.5482,68.40,41,0,8.70,LY,1638701756,
336,336,Usta Muhammad,28.1794,68.0445,81.64,14,0,4.36,PK,1638701768,
375,375,Constitución,-35.3333,-72.4167,62.44,57,0,3.22,CL,1638701773,
381,381,Isla Vista,34.4133,-119.8610,64.11,66,0,4.18,US,1638701774,
382,382,Acapulco de Juárez,16.8634,-99.8901,75.60,72,0,3.65,MX,1638701774,
417,417,Sur,22.5667,59.5289,78.37,55,0,9.73,OM,1638701778,


In [69]:
target_type = "hotel"
radius = 5000
key = gmap_key


params = {"types": target_type,
          "radius": radius,
          "key": key,
          "keyword" : "hotel"}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Retrieving Results for Index 35: Salalah.
Closest hotel is Salalah Gardens Hotel.
Retrieving Results for Index 43: Adelaide.
Closest hotel is The Terrace Hotel Adelaide.
Retrieving Results for Index 131: General Roca.
Closest hotel is Hotel Bait.
Retrieving Results for Index 168: Waddān.
Missing field/result... skipping.
Retrieving Results for Index 252: Jalu.
Closest hotel is Thuraya Tourist Hotel.
Retrieving Results for Index 336: Usta Muhammad.
Closest hotel is masah allah hotal.
Retrieving Results for Index 375: Constitución.
Closest hotel is Bancoarena Hotel.
Retrieving Results for Index 381: Isla Vista.
Closest hotel is Hampton Inn Santa Barbara/Goleta.
Retrieving Results for Index 382: Acapulco de Juárez.
Closest hotel is Hotel Krystal Beach Acapulco.
Retrieving Results for Index 417: Sur.
Closest hotel is Sur Plaza Hotel.
Retrieving Results for Index 457: Neuquén.
Closest hotel is ILLIA 121 APART HOTEL NEUQUÉN.


In [70]:
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
35,35,Salalah,17.0151,54.0924,83.66,34,0,8.37,OM,1638701726,Salalah Gardens Hotel
43,43,Adelaide,-34.9333,138.6000,67.42,61,0,4.00,AU,1638701727,The Terrace Hotel Adelaide
131,131,General Roca,-39.0333,-67.5833,61.20,53,0,7.49,AR,1638701738,Hotel Bait
168,168,Waddān,29.1614,16.1390,61.27,35,0,6.53,LY,1638701743,Waddan
252,252,Jalu,29.0331,21.5482,68.40,41,0,8.70,LY,1638701756,Thuraya Tourist Hotel
336,336,Usta Muhammad,28.1794,68.0445,81.64,14,0,4.36,PK,1638701768,masah allah hotal
375,375,Constitución,-35.3333,-72.4167,62.44,57,0,3.22,CL,1638701773,Bancoarena Hotel
381,381,Isla Vista,34.4133,-119.8610,64.11,66,0,4.18,US,1638701774,Hampton Inn Santa Barbara/Goleta
382,382,Acapulco de Juárez,16.8634,-99.8901,75.60,72,0,3.65,MX,1638701774,Hotel Krystal Beach Acapulco
417,417,Sur,22.5667,59.5289,78.37,55,0,9.73,OM,1638701778,Sur Plaza Hotel


Markers(data_bounds=[(-58.232008170471055, -179.99999), (63.5012627159256, 179.99999)], markers=[Marker(locati…

In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)




In [81]:
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))